### This notebook presents an execution tree enhanced with provenance.
### In this view, ALL of the dependencies are presented in the tree. None simplification or cleaning is performed
### The grey objects in the graph are Evaluations that are NOT function calls

In [1]:
# Imports
import sqlite3

from graphviz import Graph

from pyalgdb.validity import Validity
from pyalgdb.node import Node
from pyalgdb.execution_tree import ExecutionTree
from pyalgdb.execution_tree_creator import ExecTreeCreator
from pyalgdb.top_down import TopDown
from pyalgdb.heaviest_first import HeaviestFirst
from pyalgdb.visualization import Visualization
from pyalgdb.provenance_enhancement import ProvenanceEnhancement 
from pyalgdb.single_stepping import SingleStepping
from pyalgdb.divide_and_query import DivideAndQuery

In [2]:
NOW2_SQLITE_PATH = 'C:/Users/linha/Desktop/ws/py-scripts-examples/min-max/.noworkflow/db.sqlite'

In [3]:
CURSOR = sqlite3.connect(NOW2_SQLITE_PATH).cursor()

In [4]:
creator = ExecTreeCreator(CURSOR)
exec_tree = creator.create_exec_tree()

In [5]:
prov = ProvenanceEnhancement(exec_tree, CURSOR)

In [6]:
dependencies = prov.prov_tools.get_dependencies()

In [7]:
class CustomVisualization(Visualization):

    PROVENANCE_COLOR = 'dodgerblue'
    INVALID_COLOR = 'tomato'

    NODE_FUNC_CALL = 'beige'
    NODE_NOT_FUNC_CALL = 'grey60'

    def view_exec_tree_prov(self, graph_name, dependencies:list):
        file_name = "{}.gv".format(graph_name)
        self.graph = Graph(graph_name, filename=file_name)
        self.graph.attr('node', shape='box')
        self.graph.attr('graph', ordering='out')
        root_node = self.exec_tree.root_node
        self.graph.node(str(root_node.ev_id), root_node.name, fillcolor=self.INVALID_COLOR, style='filled')
        self.navigate(root_node)
        eval_node = self.exec_tree.node_under_evaluation
        if eval_node is not None:
            self.graph.node(str(eval_node.ev_id), str(eval_node.name), fillcolor=self.NODE_IN_EVALUATION, style='filled')
        for d in dependencies: # this loop draws the provenance links between nodes
            
            if (d.dependent.code_component_type == 'call'):
                self.graph.node(str(d.dependent.ev_id), "{} {}".format(d.dependent.ev_id,d.dependent.code_component_name), fillcolor=self.NODE_FUNC_CALL, style='filled')
            else:
                self.graph.node(str(d.dependent.ev_id), "{} {}".format(d.dependent.ev_id,d.dependent.code_component_name), fillcolor=self.NODE_NOT_FUNC_CALL, style='filled')
   
            if (d.influencer.code_component_type == 'call'):
                self.graph.node(str(d.influencer.ev_id), "{} {}".format(d.influencer.ev_id,d.influencer.code_component_name), fillcolor=self.NODE_FUNC_CALL, style='filled')
            else:
                self.graph.node(str(d.influencer.ev_id), "{} {}".format(d.influencer.ev_id,d.influencer.code_component_name), fillcolor=self.NODE_NOT_FUNC_CALL, style='filled')
    
            self.graph.edge(str(d.dependent.ev_id), str(d.influencer.ev_id), None, color=self.PROVENANCE_COLOR, dir='forward')
        self.graph.view()

In [8]:
vis = CustomVisualization(prov.exec_tree)

In [9]:
len(dependencies)

212

In [11]:
vis.view_exec_tree_prov('exec_tree_prov_full', dependencies)